In [ ]:
# Advanced CRUD Operations

In [1]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'})
]

courses_collection.bulk_write(operations)
print('Courses inserted successfully!')

Courses inserted successfully!


In [2]:
import pandas as pd

print('                         \033[1mCourses Data\033[0m')
data_courses = list(courses_collection.find({}))
df = pd.DataFrame(data_courses)
display(df)

                         Courses Data


,_id,course,enrollments,department
0,691086e267964d50940f4a3a,Math 101,30,Mathematics
1,691086e267964d50940f4a3b,CS 102,25,Computer Science
2,691086e267964d50940f4a3c,History 201,20,History
3,691086e267964d50940f4a3d,Physics 202,15,Physics


In [ ]:
# Complex Filtering and Querying

In [3]:
for course in courses_collection.find({'enrollments': {'$gt': 20}}, {'_id': 0}):
    print(course)

{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [4]:
for course in courses_collection.find({'department': {'$in' : ['Computer Science', 'Mathematics']}}, {'_id': 0}):
    print(course)

{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [ ]:
# Aggregation Framework for Data Analysis

In [5]:
pipeline = [
    {'$group': {'_id': '$department', 'avg_enrollment': {'$avg': '$enrollments'}}}
]

for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Computer Science', 'avg_enrollment': 25.0}
{'_id': 'Physics', 'avg_enrollment': 15.0}
{'_id': 'Mathematics', 'avg_enrollment': 30.0}
{'_id': 'History', 'avg_enrollment': 20.0}


In [6]:
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]

for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'History', 'max_enrollment': 20}
{'_id': 'Computer Science', 'max_enrollment': 25}
{'_id': 'Physics', 'max_enrollment': 15}
{'_id': 'Mathematics', 'max_enrollment': 30}


In [ ]:
# Data Transformation using project and addFields

In [7]:
pipeline = [
    {'$project': {'course_name': '$course', 'department_name': '$department', 'enrollments': 1, '_id': 0}}
]

for result in courses_collection.aggregate(pipeline):
    print(result)

{'enrollments': 30, 'course_name': 'Math 101', 'department_name': 'Mathematics'}
{'enrollments': 25, 'course_name': 'CS 102', 'department_name': 'Computer Science'}
{'enrollments': 20, 'course_name': 'History 201', 'department_name': 'History'}
{'enrollments': 15, 'course_name': 'Physics 202', 'department_name': 'Physics'}


In [8]:
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'}}}},
    {'$project': {'_id': 0}}
]

for result in courses_collection.aggregate(pipeline):
    print(result)

{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrollment_category': 'low'}
{'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'enrollment_category': 'low'}


In [ ]:
# Homework 1

In [9]:
pipeline = [
    {'$group': {'_id': '$department', 'course_count': {'$sum': 1}}}
]

print('\033[1mHomework 1: Count of Courses per Department\033[0m')
for result in courses_collection.aggregate(pipeline):
    print(result)

Homework 1: Count of Courses per Department
{'_id': 'Computer Science', 'course_count': 1}
{'_id': 'Physics', 'course_count': 1}
{'_id': 'Mathematics', 'course_count': 1}
{'_id': 'History', 'course_count': 1}


In [ ]:
# Homework 2

In [10]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

courses_collection.delete_many({})

operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}),
    InsertOne({'course': 'Machine Learning 103', 'enrollments': 40, 'department': 'Computer Science'}),
    InsertOne({'course': 'Web Development 104', 'enrollments': 35, 'department': 'Computer Science'}),
]

courses_collection.bulk_write(operations)
print('Courses inserted successfully!')

Courses inserted successfully!


In [11]:
pipeline = [ 
    {'$match': {'department': 'Computer Science', 'enrollments': {'$gt': 25}}},
    {'$group': {'_id': '$department', 'high_enrollment_courses': {'$push': '$course'}}}
]

print('\033[1mHomework 2: Filter n Group Courses with Enrollments > 25\033[0m')
for result in courses_collection.aggregate(pipeline):
    print(result)

Homework 2: Filter n Group Courses with Enrollments > 25
{'_id': 'Computer Science', 'high_enrollment_courses': ['Machine Learning 103', 'Web Development 104']}


In [ ]:
# Homework 3

In [14]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
students_collection = db['students']

students_data = [
    InsertOne({'name': 'Jack', 'gender': 'M', 'course': 'Math 101'}),
    InsertOne({'name': 'Joe', 'gender': 'M', 'course': 'CS 102'}),
    InsertOne({'name': 'Shelly', 'gender': 'F', 'course': 'CS 102'}),
    InsertOne({'name': 'Owen', 'gender': 'M', 'course': 'Physics 202'}),
    InsertOne({'name': 'Liam', 'gender': 'M', 'course': 'Web Development 104'}),
    InsertOne({'name': 'Noah', 'gender': 'M', 'course': 'Physics 202'}),
    InsertOne({'name': 'Ava', 'gender': 'F', 'course': 'Machine Learning 103'}),
    InsertOne({'name': 'Sophie', 'gender': 'F', 'course': 'History 201'}),
    InsertOne({'name': 'Emma', 'gender': 'F', 'course': 'History 201'}),
    InsertOne({'name': 'Jesse', 'gender': 'M', 'course': 'Math 101'})
]

students_collection.bulk_write(students_data)
print('Students have been successfully enrolled in courses!')

Students have been successfully enrolled in courses!


In [15]:
import pandas as pd

print('                \033[1mStudents in Course\033[0m')
students_data = list(students_collection.find({}))
df = pd.DataFrame(students_data)
display(df)

                Students in Course


,_id,name,gender,course
0,6910877167964d50940f4a51,Jack,M,Math 101
1,6910877167964d50940f4a52,Joe,M,CS 102
2,6910877167964d50940f4a53,Shelly,F,CS 102
3,6910877167964d50940f4a54,Owen,M,Physics 202
4,6910877167964d50940f4a55,Liam,M,Web Development 104
5,6910877167964d50940f4a56,Noah,M,Physics 202
6,6910877167964d50940f4a57,Ava,F,Machine Learning 103
7,6910877167964d50940f4a58,Sophie,F,History 201
8,6910877167964d50940f4a59,Emma,F,History 201
9,6910877167964d50940f4a5a,Jesse,M,Math 101


In [18]:
pipeline = [
    {'$lookup': {'from': 'students', 'localField': 'course', 'foreignField': 'course','as': 'enrolled_students'}},
    {'$project': {'_id': 0, 'course': 1, 'department': 1, 'enrollments': 1, 'enrolled_students.name': 1}}
]

print('\033[1mHomework 3: Join Courses with Students\033[0m')
for result in courses_collection.aggregate(pipeline):
    print(result)

Homework 3: Join Courses with Students
{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'enrolled_students': [{'name': 'Jack'}, {'name': 'Jesse'}]}
{'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'enrolled_students': [{'name': 'Joe'}, {'name': 'Shelly'}]}
{'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrolled_students': [{'name': 'Sophie'}, {'name': 'Emma'}]}
{'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'enrolled_students': [{'name': 'Owen'}, {'name': 'Noah'}]}
{'course': 'Machine Learning 103', 'enrollments': 40, 'department': 'Computer Science', 'enrolled_students': [{'name': 'Ava'}]}
{'course': 'Web Development 104', 'enrollments': 35, 'department': 'Computer Science', 'enrolled_students': [{'name': 'Liam'}]}


In [19]:
import pandas as pd

pipeline = [
    {'$lookup': {'from': 'students', 'localField': 'course', 'foreignField': 'course','as': 'enrolled_students'}},
    {'$project': {'_id': 0, 'course': 1, 'department': 1, 'enrollments': 1, 'enrolled_students.name': 1}}
]

result = courses_collection.aggregate(pipeline)
df = pd.DataFrame(result)
print('\033[1mHomework 3: Join Courses with Students\033[0m')
pd.set_option('display.max_colwidth', 100)
display(df)

Homework 3: Join Courses with Students


,course,enrollments,department,enrolled_students
0,Math 101,30,Mathematics,"[{'name': 'Jack'}, {'name': 'Jesse'}]"
1,CS 102,25,Computer Science,"[{'name': 'Joe'}, {'name': 'Shelly'}]"
2,History 201,20,History,"[{'name': 'Sophie'}, {'name': 'Emma'}]"
3,Physics 202,15,Physics,"[{'name': 'Owen'}, {'name': 'Noah'}]"
4,Machine Learning 103,40,Computer Science,[{'name': 'Ava'}]
5,Web Development 104,35,Computer Science,[{'name': 'Liam'}]
